In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import statsmodels.api as sm
import os
import sys

sys.path.insert(0, os.path.abspath('../developer'))

from config import MOCK_DATA, CODE, OUT
from developer.utilities import read_yaml
from developer.analysis.model import load_model
import re

In [10]:
df_total = pd.read_csv(OUT / "data" / "data_regression.csv")

##### regression

In [5]:
# Select the design matrix (explanatory variables) and the outcome variables
outcome = 'utility'
explanatory_vars = [col for col in df_total.columns if "att" in col]

X = df_total[explanatory_vars].astype(int)
y = df_total[outcome].astype(int)

In [6]:
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

In [7]:
print("Regression results for Attribute A:")
print(model.summary())

Regression results for Attribute A:
                            OLS Regression Results                            
Dep. Variable:                utility   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     1.144
Date:                Mon, 07 Aug 2023   Prob (F-statistic):              0.317
Time:                        12:38:38   Log-Likelihood:                -218.23
No. Observations:                 132   AIC:                             478.5
Df Residuals:                     111   BIC:                             539.0
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------

## Plots

##### 1.1 Relative support plot

In [9]:
import plotly.graph_objects as go

# Assuming 'model' is the variable that holds your regression results
# Extract the coefficients and standard errors for each attribute level
att_1_levels = ['PhaseOut', 'Stop&Reduce', 'Stop&Maintain', 'StatusQuo']
att_2_levels = ['HighInvestment&Int', 'HighInvestment&Int&Consideration', 'LowInvestment&LowConsideration', 'LowInvestment']
att_3_levels = ['HealthEdu', 'EnergyAccess', 'LowPrices', 'Transfers', 'NothingSoc']
att_4_levels = ['CreateJobs', 'EarlyPension', 'JobGuarantee', 'Retrain', 'NothingEco']
att_5_levels = ['CivilNGO', 'EnergySector', 'LabourUnion', 'LocalGov', 'Media', 'Researchers', 'CentralGov']

att_levels = [att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

att_colors = ['red', 'blue', 'green', 'orange', 'purple']  # Colors for each attribute group

fig = go.Figure()

total_levels = sum(len(levels) for levels in att_levels)

# Loop through each attribute group and add the data to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model.bse[f'att_{5-i}_{level}'] for level in levels]

    relative_differences = [coeff - att_coefficients[-1] for coeff in att_coefficients]

    fig.add_trace(go.Scatter(
        x=relative_differences,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color='darkgray', size=10),
        orientation='h',
        showlegend = False,
    ))

    fig.add_shape(
        type="rect",
        x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
        x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
        y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
        y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
        fillcolor=att_colors[i],
        opacity=0.1,  # Set the opacity for a light transparent effect
        layer="below",  # Place the rectangle below the scatter plot markers
    )

# Add a vertical line at x=0 for reference
fig.add_shape(type="line", x0=0, x1=0, y0=att_5_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

# Update the layout of the error bar plot
fig.update_layout(
    title='Relative Rating Differences',
    xaxis_title='',
    yaxis_title='Attribute Levels',
    yaxis=dict(categoryorder='array', categoryarray=att_5_levels),  # Set the categoryorder for y-axis based on att_1_levels
    xaxis=dict(tickformat='.2f', zeroline=False),  # Remove x-axis zeroline
    showlegend=True,  # Show legend with attribute names
    margin=dict(l=80, r=30, b=40, t=80),
    height=600,  # Set the height of the plot to 600 pixels
    width=1000,
    title_x=0.62,
)

# Show the interactive error bar plot
fig.show()
